In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import requests
from datetime import datetime
from tqdm import tqdm

/Users/elias/Desktop/PTSFC/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
df = get_energy_data()

  0%|          | 0/257 [00:00<?, ?it/s]/var/folders/c7/nx3f8qlj38ld0jj0l_0nyp2h0000gn/T/ipykernel_55440/1783941483.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 257/257 [00:55<00:00,  4.66it/s]


In [5]:
df.tail()

,Netzlast_Gesamt
date_time,
2023-11-22 10:00:00,68283.75
2023-11-22 11:00:00,69746.25
2023-11-22 12:00:00,69942.25
2023-11-22 13:00:00,68124.00
2023-11-22 14:00:00,16855.75


Rename column for convenience

In [6]:
df = df.rename(columns={"Netzlast_Gesamt": "gesamt"})

Rescale Netzlast so it fits requirements

In [7]:
df['gesamt'] = df['gesamt'] / 1000

Check dtypes and if columns contain and missing values

In [8]:
df.dtypes

gesamt    float64
dtype: object

In [9]:
df.isna().any()

gesamt    False
dtype: bool

Define weekday column

In [10]:
df["weekday"] = df.index.weekday #Monday=0, Sunday=6
#df["time"] = df.index.strftime("%H:%M")

In [11]:
# from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
# 
# en_df = get_energy_data()
# df = prepare_data(en_df)

Lead times are

In [12]:
horizons_def = [36, 40, 44, 60, 64, 68]#[24 + 12*i for i in range(5)]
horizons_def

[36, 40, 44, 60, 64, 68]

Adapt horzions so they actually fit

In [13]:
horizons = [h+1 for h in horizons_def]
horizons

[37, 41, 45, 61, 65, 69]

In [14]:
def get_date_from_horizon(last_ts, horizon):
    return last_ts + pd.DateOffset(hours=horizon)

In [15]:
LAST_IDX = -1
LAST_DATE = df.iloc[LAST_IDX].name

Get time and date that correspond to the lead times (starting at the last observation in our data which should be the respective thursday 0:00)  
*Attention*: if the last timestamp in the data is not thursday 0:00, you have to adjust your lead times accordingly

In [16]:
horizon_date = [get_date_from_horizon(LAST_DATE, h) for h in horizons]
horizon_date

[Timestamp('2023-11-24 03:00:00'),
 Timestamp('2023-11-24 07:00:00'),
 Timestamp('2023-11-24 11:00:00'),
 Timestamp('2023-11-25 03:00:00'),
 Timestamp('2023-11-25 07:00:00'),
 Timestamp('2023-11-25 11:00:00')]

quantile levels

In [17]:
tau = [.025, .25, .5, .75, .975]

In [18]:
#rows correspond to horizon, columns to quantile level
pred_baseline = np.zeros((6,5))

In [19]:
last_t = 100
for i,d in enumerate(horizon_date):
    
    weekday = d.weekday()
    hour = d.hour
    
    df_tmp = df.iloc[:LAST_IDX]
    
    cond = (df_tmp.weekday == weekday) & (df_tmp.index.time == d.time())
    
    pred_baseline[i,:] = np.quantile(df_tmp[cond].iloc[-last_t:]["gesamt"], q=tau)
    

In [20]:
pred_baseline

array([[38.68145   , 41.425625  , 43.776375  , 46.582     , 55.19215625],
       [46.88518125, 55.7975    , 59.27475   , 63.3901875 , 71.7006125 ],
       [53.63964375, 61.140375  , 64.128     , 67.403125  , 75.8170625 ],
       [35.877375  , 38.225     , 40.78025   , 44.3040625 , 50.88774375],
       [40.6279    , 42.6660625 , 45.58475   , 48.5829375 , 54.6359625 ],
       [47.84225625, 51.6094375 , 54.42825   , 57.365625  , 63.6835375 ]])

Visually check if quantiles make sense

In [ ]:
x = horizons
_ = plt.plot(x,pred_baseline, ls="", marker="o", c="black")
_ = plt.xticks(x, x)
_ = plt.plot((x,x),(pred_baseline[:,0], pred_baseline[:,-1]),c='black')

In [ ]:
from datetime import datetime, date, timedelta
date_str = datetime.today().strftime('%Y%m%d')

In [ ]:
date_str = date.today() #- timedelta(days=1)
date_str = date_str.strftime('%Y-%m-%d')
date_str

In [ ]:
df_sub = pd.DataFrame({
    "forecast_date": date_str,
    "target": "energy",
    "horizon": [str(h) + " hour" for h in horizons_def],
    "q0.025": pred_baseline[:,0],
    "q0.25": pred_baseline[:,1],
    "q0.5": pred_baseline[:,2],
    "q0.75": pred_baseline[:,3],
    "q0.975": pred_baseline[:,4]})
df_sub

In [ ]:
#need to change this
PATH = "/save/to/path"


df_sub.to_csv(PATH+date_str+"_power_benchmark.csv", index=False)